In [ ]:
# --quiet suprime o output de texto, para manter o notebook mais legível
!pip install --quiet 'crewai==0.28.8'
!pip install --quiet 'crewai_tools==0.1.6'
!pip install --quiet 'langchain-community'
!pip install --quiet 'duckduckgo-search>=6.1.7'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 977.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 9.8 MB/s eta

In [ ]:
from crewai_tools import FileReadTool, ScrapeWebsiteTool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

ModuleNotFoundError: No module named 'crewai_tools'

In [ ]:
# Cria a ferramenta a ser usada para buscar na internet
search_tool = DuckDuckGoSearchRun()
# Cria a ferramenta a ser usada para obter mais informações sobre um site em específico
scrape_tool = ScrapeWebsiteTool()

In [ ]:
from crewai import Agent, Task, Crew
import os

os.environ['OPENAI_API_KEY'] = ''
os.environ['OPENAI_MODEL_NAME'] = 'gpt3.5-turbo'

gpt35 = ChatOpenAI(
  temperature=0.7,
  model_name="gpt-3.5-turbo",
)

# Cria um agente que vai buscar e reunir informações empresariais sobre a empresa a ser coberta.
# Define seu objetivo e backstory tal que possa atender à tarefa. O mesmo vai também ser feito para todos os outros agentes.
company_research_agent = Agent(
    role="Company Research Agent",
    goal="Gather detailed information about the target company, including their business, products, services, and market position.",
    backstory=(
        "Tasked with collecting comprehensive data about potential clients to inform the sales pitch."
    ),
    allow_delegation=True,
    verbose=True,
    llm=gpt35
)

# Define a tarefa esperada do agente, incluindo detalhamento sobre qual o output esperado.
# O mesmo vai também ser feito para todos os outros agentes.
company_research_task = Task(
    description=(
        "Collect comprehensive data about {company_name} from their website ({company_website}) and other sources. "
        "Include details about their business, products, services, and market position."
    ),
    expected_output=(
        "Detailed report on {company_name}'s business, products, services, and market position."
    ),
    agent=company_research_agent,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# Cria um agente que vai analisar com detalhe o website da empresa, buscand por melhoras possíveis.
website_analysis_agent = Agent(
    role="Website Analysis Agent",
    goal="Analyze the target company's website for areas of improvement, including design, performance, and user experience.",
    backstory=(
        "Focuses on identifying specific issues and opportunities for enhancement on the client's website."
    ),
    allow_delegation=True,
    verbose=True,
    llm=gpt35
)

# Define a tarefa esperada do agente, incluindo detalhamento sobre qual o output esperado.
website_analysis_task = Task(
    description=(
        "Analyze {company_name}'s website ({company_website}) to identify areas of improvement in design, performance, and user experience."
    ),
    expected_output=(
        "Detailed analysis report highlighting areas of improvement for {company_name}'s website, including design, performance, and user experience."
    ),
    agent=website_analysis_agent,
    tools=[scrape_tool, search_tool]
)

In [ ]:
# Cria um agente que vai analisar o conteúdo do site, de um ponto de vista de marketing, buscando formas de melhorar engajamento e conversão.
content_strategy_agent = Agent(
    role="Content Strategy Agent",
    goal="Develop content strategies and identify how improved content can drive engagement and conversions.",
    backstory=(
        "Specializes in creating a compelling narrative and strategy for content improvements on the client's website."
    ),
    allow_delegation=True,
    verbose=True,
    llm=gpt35
)

# Define a tarefa esperada do agente, incluindo detalhamento sobre qual o output esperado.
content_strategy_task = Task(
    description=(
        "Develop a content strategy for {company_name}'s website ({company_website}) to enhance user engagement and conversion rates."
    ),
    expected_output=(
        "Comprehensive content strategy for {company_name}'s website, including recommendations for content improvements."
    ),
    agent=content_strategy_agent,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# Cria um agente que vai gerar um sales pitch ("discurso de vendas") usando as pesquisas dos outros agentes.
# Tem o intuito de convencer a companhia alvo que seu website precisa ser modificado com a nossa companhia "Code With Prince",
#  com base no que foi encontrado pelos demais agentes.
sales_pitch_agent = Agent(
    role="Sales Pitch Agent",
    goal="Create a personalized and persuasive sales pitch to convince the target company to have their website modified by 'Code With Prince'.",
    backstory=(
        "Combines research, analysis, and content strategy to craft a compelling sales pitch."
    ),
    allow_delegation=False,
    verbose=True,
    llm=gpt35
)

# Define a tarefa esperada do agente, incluindo detalhamento sobre qual o output esperado.
sales_pitch_task = Task(
    description=(
        "Combine the research, analysis, and content strategy to create a personalized and persuasive sales pitch for {company_name} "
        "to convince them to have their website modified by 'Code With Prince'."
    ),
    expected_output=(
        "Customized sales pitch for {company_name}, highlighting the benefits of having their website modified by 'Code With Prince'."
    ),
    agent=sales_pitch_agent,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# Inicializa a Crew (representa o grupo de agente, quais as tasks deles e como devem colaborar).
crew = Crew(
    agents=[
        company_research_agent,
        website_analysis_agent,
        content_strategy_agent,
        sales_pitch_agent
    ],
    tasks=[
        company_research_task,
        website_analysis_task,
        content_strategy_task,
        sales_pitch_task
    ],
    verbose=2,
    memory=True,
    cache=True,
    llm=gpt35
)

In [ ]:
# Define a companhia alvo a ser analisada, e o seu site.
inputs = {
    "company_name": "CrewAI",
    "company_website": "https://crewai.com"
}

# Começa o trabalho da Crew.
result = crew.kickoff(inputs=inputs)

In [ ]:
# Usa Markdown para mostrar a resposta gerada.
from IPython.display import Markdown

# Armazena o resultado em um arquivo.
with open("result.md", "w") as file:
    file.write(result)

Markdown(result)

Dear CrewAI Team,

After analyzing your website and understanding your current needs, I believe that Code With Prince can help take your online platform to the next level. Here are the key benefits of having your website modified by our team:

1. Visual Elements: We will incorporate more engaging graphics and images to captivate your users' attention and enhance their overall experience on the platform.

2. Performance Optimization: Our team will implement lazy loading for images and optimize your website's code to ensure faster loading speeds and improved performance, leading to a seamless user experience.

3. User Experience Enhancements: By adding customer testimonials and a live chat feature, we aim to build trust with your users and provide quick customer support, ultimately increasing user engagement.

4. Continuous Improvement: Through user testing and specific metrics analysis, we will continuously gather feedback to enhance user engagement and conversion rates, ensuring the success of the implemented strategies.

5. Timeline and Budget: We will provide a clear timeline for implementing our content strategy and estimate the budget for design and performance optimizations, ensuring high-quality results within your specified timeframe and budget.

With Code With Prince's expertise in website modifications and our commitment to achieving outstanding results, we are confident that we can help elevate CrewAI's website to new heights. We look forward to the opportunity to work with your team and create a more compelling user experience for your visitors.

Best regards,
[Your Name]
Code With Prince Team